In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.utils.np_utils import to_categorical  
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.utils.np_utils import to_categorical 
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_set_0 = pd.read_csv('0.csv')    # reading the csv file
train_set_0.head()
train_set_1 = pd.read_csv('1.csv')    # reading the csv file
train_set_1.head()

,Unnamed: 0,image_name,target
0,0,ISIC_0000002.jpg.jpg,1
1,1,ISIC_0000004.jpg.jpg,1
2,2,ISIC_0000026.jpg.jpg,1
3,3,ISIC_0000029.jpg.jpg,1
4,4,ISIC_0000030.jpg.jpg,1


In [3]:
from tqdm import tqdm
from keras.preprocessing import image
from PIL import Image
import tensorflow as tf

train_image = []

for i in tqdm(range(train_set_0.shape[0])):
    img = tf.keras.utils.load_img('0/' + train_set_0['image_name'][i],target_size=(224,224,3))
    img = tf.keras.utils.img_to_array(img)
    img = img/255
    train_image.append(img)

X = np.array(train_image)

100%|██████████████████████████████████████| 2233/2233 [00:03<00:00, 686.13it/s]


In [4]:
from tqdm import tqdm
from keras.preprocessing import image
from PIL import Image
import tensorflow as tf

train_image = []

for i in tqdm(range(train_set_1.shape[0])):
    img = tf.keras.utils.load_img('1/' + train_set_1['image_name'][i],target_size=(224,224,3))
    img = tf.keras.utils.img_to_array(img)
    img = img/255
    train_image.append(img)

Y = np.array(train_image)

100%|██████████████████████████████████████| 1872/1872 [00:02<00:00, 665.93it/s]


In [5]:
y_0=train_set_0['target']
y_1=train_set_1['target']



In [6]:
from sklearn.model_selection import train_test_split
X_0_train, X_0_test, y_0_train, y_0_test = train_test_split(X,y_0, train_size=0.7,random_state = 42, shuffle = True)
X_1_train, X_1_test, y_1_train, y_1_test = train_test_split(Y,y_1, train_size=0.7,random_state = 42, shuffle = True)

In [7]:
X_train = np.concatenate((X_0_train,X_1_train))
y_train = np.concatenate((y_0_train,y_1_train))
X_test = np.concatenate((X_0_test,X_1_test))
y_test= np.concatenate((y_0_test,y_1_test))

In [8]:
X_train_x, X_valid, y_train_x, y_valid = train_test_split(X_train,y_train, test_size=0.15,random_state = 42, shuffle = True)

In [9]:
print(X_valid.shape), print(y_valid.shape)
print(X_train_x.shape), print(y_train_x.shape)

(431, 224, 224, 3)
(431,)
(2442, 224, 224, 3)
(2442,)


(None, None)

In [10]:
datagen = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range 
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally 
        height_shift_range=0.1,  # randomly shift images vertically 
        ) 

datagen.fit(X_train_x)

In [11]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, MaxPool2D
from keras.layers.core import Lambda
from keras.layers import *
from keras.models import Model
import keras.backend as K
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import datasets,models, losses
from keras.layers import Conv2D, MaxPool2D, Input, ZeroPadding2D
import os

def Global_attention_block(inputs):
    
    shape=K.int_shape(inputs)
    x=AveragePooling2D(pool_size=(shape[1],shape[2])) (inputs)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('relu') (x)
    x=Conv2D(shape[3],1, padding='same') (x)
    x=Activation('sigmoid') (x)
    C_A=Multiply()([x,inputs])
    
    x=Lambda(lambda x: K.mean(x,axis=-1,keepdims=True))  (C_A)
    x=Activation('sigmoid') (x)
    S_A=Multiply()([x,C_A])
    
    return S_A

In [12]:
def define_model():
    img_inputs = keras.Input(shape=(224, 224, 3))

    conv_0 = Conv2D(16,3,strides=(1, 1), padding='same')
    x = conv_0(img_inputs) 
    max_00 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(x)
    
    conv_0 = Conv2D(16,3,strides=(1, 1), padding='same')(max_00)
    max_0 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(conv_0)
    
    conv_1 = Conv2D(32,1,strides=(1, 1), padding='same')(max_0)
    bn = tf.keras.layers.BatchNormalization()(conv_1)
    act_1 = tf.keras.layers.ReLU()(bn)
    max_1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_1)

    conv_2 = Conv2D(32,3,strides=(1, 1), padding='same')(max_0)
    bn = tf.keras.layers.BatchNormalization()(conv_2)
    act_2 = tf.keras.layers.ReLU()(bn)
    max_2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_2)

    conv_3 = Conv2D(32,3,strides=(1, 1), padding='same')(max_0)
    bn = tf.keras.layers.BatchNormalization()(conv_2)
    act_3 = tf.keras.layers.ReLU()(bn)

    conv_3 = Conv2D(32,3,strides=(1, 1), padding='same')(act_3)
    bn = tf.keras.layers.BatchNormalization()(conv_3)
    act_3_1 = tf.keras.layers.ReLU()(bn)
    max_3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_3_1)


    conv_4 = Conv2D(32,3,strides=(1, 1), padding='same')(max_0)
    bn = tf.keras.layers.BatchNormalization()(conv_4)
    act_4 = tf.keras.layers.ReLU()(bn)
    max_4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_4)
    
    conc_1 = tf.keras.layers.Concatenate(axis = -1)([max_2,max_3])
    
    conc_1 = Conv2D(32,1,strides=(1, 1), padding='same')(conc_1)
    bn = tf.keras.layers.BatchNormalization()(conc_1)
    act_conc_1 = tf.keras.layers.ReLU()(bn)

    conc_2 = tf.keras.layers.Concatenate(axis = -1)([max_1,act_conc_1,max_4])
    
    conv_5 = Conv2D(32,1,strides=(1, 1), padding='same')(conc_2)
    bn = tf.keras.layers.BatchNormalization()(conv_5)
    act_conv_3 = tf.keras.layers.ReLU()(bn)
    max_5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_conv_3)
    
    conv_6 = Conv2D(32,1,strides=(1, 1), padding='same')(max_5)
    bn = tf.keras.layers.BatchNormalization()(conv_6)
    act_conv_3 = tf.keras.layers.ReLU()(bn)
    max_6 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_conv_3)
    
    """conv_7 = Conv2D(32,1,strides=(1, 1), padding='same')(max_6)
    bn = tf.keras.layers.BatchNormalization()(conv_7)
    act_conv_3 = tf.keras.layers.ReLU()(bn)
    max_7 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding = 'same')(act_conv_3)"""
    
    model=Model(inputs=img_inputs,outputs=max_6)
    return model

    

In [13]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import datasets,models, losses
from keras.layers import Conv2D, MaxPool2D, Input, ZeroPadding2D


image_size = 224

model1=tf.keras.applications.DenseNet121(include_top=False,
                       weights='imagenet',
                       input_shape=(image_size,image_size,3))

lr2 = 0.0001
loss_fun= 'sparse_categorical_crossentropy'

model1_in = model1.input
model1_out = model1.output

model2 = define_model()

model2_in = model2.input
model2_out = model2.output


conc =tf.keras.layers.Concatenate()([model1_out, model2_out])

attend_feature_1 = Global_attention_block(conc)
base_out=GlobalAveragePooling2D()(attend_feature_1)

base_out=GlobalAveragePooling2D()(base_out)
x = Dense(2,activation='sigmoid')(base_out)

model=Model(inputs=[model1.input,model2.input],outputs=x)


filepath = "DesneNet+parallel_32.hdf5"
lr_decay=ReduceLROnPlateau(monitor='loss', factor=0.8, patience=3, verbose=1)
checkpoint = ModelCheckpoint(filepath, monitor='acc',
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')
for layer in model.layers:
    layer.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr2,decay=0.00001),
              loss=loss_fun,
              metrics=['acc'])

model.summary()

2022-07-07 20:18:29.785744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1922] Ignoring visible gpu device (device: 1, name: GeForce GT 710, pci bus id: 0000:b3:00.0, compute capability: 3.5) with core count: 1. The minimum required count is 8. You can adjust this requirement with the env var TF_MIN_GPU_MULTIPROCESSOR_COUNT.
2022-07-07 20:18:29.786122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-07 20:18:30.499194: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 155 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:65:00.0, compute capability: 7.0
2022-07-07 20:18:30.506455: I tensorflow/stream_executor/cu

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                            

 n)                                                                                               
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 56, 56, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 56, 56, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 conv3_block2_0_relu (Activatio  (None, 28, 28, 160)  0          ['conv3_block2_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 28, 28, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 28, 28, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 28, 28, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_block7_0_bn (BatchNormal  (None, 28, 28, 320)  1280       ['conv3_block6_concat[0][0]']    
 ization) 

...

                                                                                                  
 conv5_block4_1_relu (Activatio  (None, 7, 7, 128)   0           ['conv5_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block4_2_conv (Conv2D)   (None, 7, 7, 32)     36864       ['conv5_block4_1_relu[0][0]']    
                                                                                                  
 conv5_block4_concat (Concatena  (None, 7, 7, 640)   0           ['conv5_block3_concat[0][0]',    
 te)                                                              'conv5_block4_2_conv[0][0]']    
                                                                                                  
 conv5_block5_0_bn (BatchNormal  (None, 7, 7, 640)   2560        ['conv5_block4_concat[0][0]']    
 ization) 

 te)                                                              'conv5_block8_2_conv[0][0]']    
                                                                                                  
 conv5_block9_0_bn (BatchNormal  (None, 7, 7, 768)   3072        ['conv5_block8_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block9_0_relu (Activatio  (None, 7, 7, 768)   0           ['conv5_block9_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block9_1_conv (Conv2D)   (None, 7, 7, 128)    98304       ['conv5_block9_0_relu[0][0]']    
                                                                                                  
 conv5_blo

                                                                                                  
 conv5_block13_1_conv (Conv2D)  (None, 7, 7, 128)    114688      ['conv5_block13_0_relu[0][0]']   
                                                                                                  
 conv5_block13_1_bn (BatchNorma  (None, 7, 7, 128)   512         ['conv5_block13_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block13_1_relu (Activati  (None, 7, 7, 128)   0           ['conv5_block13_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block13_2_conv (Conv2D)  (None, 7, 7, 32)     36864       ['conv5_block13_1_relu[0][0]']   
          

 concatenate (Concatenate)      (None, 28, 28, 64)   0           ['max_pooling2d_3[0][0]',        
                                                                  'max_pooling2d_4[0][0]']        
                                                                                                  
 conv2d_6 (Conv2D)              (None, 56, 56, 32)   4640        ['max_pooling2d_1[0][0]']        
                                                                                                  
 conv5_block15_1_relu (Activati  (None, 7, 7, 128)   0           ['conv5_block15_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 batch_normalization (BatchNorm  (None, 56, 56, 32)  128         ['conv2d_2[0][0]']               
 alization)                                                                                       
          

 alAveragePooling2D)                                                                              
                                                                                                  
 dense (Dense)                  (None, 2)            2114        ['global_average_pooling2d[0][0]'
                                                                 ]                                
                                                                                                  
Total params: 7,068,722
Trainable params: 6,984,562
Non-trainable params: 84,160
__________________________________________________________________________________________________


In [ ]:


history = model.fit([X_train_x,X_train_x],y_train_x,
                    validation_data=([X_valid,X_valid],y_valid),
                    epochs=70,
                    batch_size=32,
                    callbacks=[lr_decay,checkpoint])



In [ ]:
score = model.evaluate([X_test,X_test],y_test,batch_size = 32)
print("Hyperparameters : {} \n".format(model.optimizer.get_config()))
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
#Plotting the graph of accuracy VS epoch
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

#Plotting the graph of loss VS epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'],loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn
B = np.reshape(y_test, (-1, 2))
Y_pred = model.predict([X_test,X_test], 240 // 4)

#print(type(Y_pred))
print(y_test)  
y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
print('Confusion Matrix')
matrix = confusion_matrix(y_test, y_pred)

print(confusion_matrix(y_test, y_pred))
print('Classification Report')
target_names = ['non-melanoma', 'melanoma']
print(classification_report(y_test, y_pred, target_names=target_names))

plt.figure(figsize = (10,7))
sn.heatmap(matrix,annot=True,fmt='d')
plt.xlabel('Predicted value')
plt.ylabel('Truth')  

In [ ]:
pip install plot-metric

In [ ]:
from plot_metric.functions import BinaryClassification
# Visualisation with plot_metric
y_true = y_test
y_probas = y_pred
bc = BinaryClassification(y_true, y_probas, labels=['non-melanoma', 'melanoma'])

# Figures
plt.figure(figsize=(5,5))
bc.plot_roc_curve()
plt.show()